In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_path = "./emotion_model"  # 你的模型保存路径

# 加载模型和 tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.eval()  # 推理模式


e:\demo\EmotionApp\backend\EmotionAppBackend\Algorithm\HuggingFace\hf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [2]:
import torch.nn.functional as F

# 五个标签顺序，需与你训练时的 label 顺序一致！
id2label = {0: "very negative", 1: "negative", 2: "neutral", 3: "positive", 4: "very positive"}

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()
        confidence = probs[0][predicted_class].item()
    
    label = id2label[predicted_class]
    return label, confidence, probs[0]


In [3]:
text = "I am very happy with how things turned out!"
label, confidence, all_probs = predict(text)

print(f"情绪等级: {label}")
print(f"预测置信度（强度）: {confidence:.2f}")
print(f"全部情绪强度分布:")
for i, prob in enumerate(all_probs):
    print(f"{id2label[i]}: {prob:.2f}")


情绪等级: positive
预测置信度（强度）: 0.91
全部情绪强度分布:
very negative: 0.00
negative: 0.00
neutral: 0.03
positive: 0.91
very positive: 0.05
